Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [6]:
# imports
import pandas as pd
import numpy as np
import os # use this to access your environment variables
import requests # this will be used to call the APIs

In [18]:
bike_stations = pd.read_csv("../data/bike_stations_sat_3pm_clean.csv")

In [20]:
bike_stations

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18,1,94.74,"43.665269,-79.319796"
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1,14,6.67,"43.67142,-79.445947"
2,a09c67c0b419654d907c9134b108e328,Queen St E / Rhodes Ave,43.666224,-79.317693,12,11,52.17,"43.666224,-79.317693"
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5,32,13.51,"43.653236,-79.376716"
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,3,32,8.57,"43.663722,-79.380288"
...,...,...,...,...,...,...,...,...
821,9be5f078a1ed47fc11cd3cee45260f63,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),43.741906,-79.271819,1,10,9.09,"43.741906,-79.271819"
822,4ae37f3bddfb819954a15143d277dbd9,Eglinton Ave E / Brimley Rd,43.736953,-79.247984,8,11,42.11,"43.736953,-79.247984"
823,e7968ab22d9a15db0673f463144428eb,College Park South,43.659457,-79.382365,7,11,38.89,"43.659457,-79.382365"
824,62acc308c0f93ff09d28e06c73afc3ec,165 McRae Dr,43.705875,-79.368006,4,11,26.67,"43.705875,-79.368006"


In [ ]:
test

In [ ]:
# Json normalize!!!

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.